# Example code to convert VARS data to DwC

Diana LaScala-Gruenewald, March 2022

In [24]:
## Imports

import pandas as pd
import numpy as np

import re # for extracting logon info from text file

import jaydebeapi # for connecting to VARS db
import VARS # for connecting to VARS db

from datetime import datetime # for handling dates
import pytz # for handling time zones

import requests
import WoRMS # functions for querying WoRMS REST API

## Get data from VARS

In [3]:
## Extract logon information from text file

# Get list of each line in file
filename = 'VARS_logon_info.txt'
f = open(filename, 'r')
lines = f.readlines()
f.close()

# Function for extracting information from lines
def get_single_quoted_text(s):
    """ 
    Takes string s and returns any text in s that is between the first set of single quotes, removing whitespace. 
    
    Example:
    s = "What if there's more ' than one' sest of single' quotes?"
    get_single_quoted_text(s) --> 's more'
    
    """
    
    extracted_text = re.search('''(?<=')\s*[^']+?\s*(?=')''', s)
    return(extracted_text.group().strip())

# Assign logon info
dr = get_single_quoted_text(lines[2])
name = get_single_quoted_text(lines[3])
pw = get_single_quoted_text(lines[4])
un = get_single_quoted_text(lines[5])
url = get_single_quoted_text(lines[6])

In [4]:
## Build SQL query

sql = """
        SELECT index_recorded_timestamp,
               observation_uuid,
               concept,
               observation_group,
               observer,
               image_url,
               depth_meters,
               latitude,
               longitude,
               oxygen_ml_per_l,
               psi,
               salinity,
               temperature_celsius,
               video_uri,
               video_sequence_name,
               chief_scientist
        FROM annotations a
        WHERE NOT EXISTS (
           SELECT DISTINCT observation_uuid
           FROM annotations b
           WHERE (
             (  -- Delete last 2 years of annotations
             index_recorded_timestamp > DATEADD([year], - 2, GETDATE()) OR
             index_recorded_timestamp IS NULL OR
             index_recorded_timestamp < CAST('1970-01-02' AS datetime)
             )
           OR ( -- Delete embargoes by dive
             dive_number IN ('Ventana 50', 'Ventana 217', 'Ventana 218', 'Ventana 248')
              )
           OR (
             dive_number IN ('Tiburon 1001', 'Tiburon 1029', 'Tiburon 1030', 'Tiburon 1031', 'Tiburon 1032', 'Tiburon 1033', 'Tiburon 1034')
             )
           OR ( -- Delete embargoes by selectedConcept
             concept IN (
                 'Aegina sp. 1',
                 'Ctenophora',
                 'Cydippida 2',
                 'Cydippida',
                 'Intacta',
                 'Llyria',
                 'Lyrocteis',
                 'Lyroctenidae',
                 'Mertensia',
                 'Mertensiidae sp. A',
                 'Mystery Mollusc',
                 'Mystery Mollusc',
                 'Physonectae sp. 1',
                 'Platyctenida sp. 1',
                 'Platyctenida',
                 'Thalassocalycida sp. 1',
                 'Thalassocalycida',
                 'Thliptodon sp. A',
                 'Tjalfiella tristoma',
                 'Tjalfiella',
                 'Tjalfiellidae',
                 'Tuscarantha braueri',
                 'Tuscarantha luciae',
                 'Tuscarantha',
                 'Tuscaretta globosa',
                 'Tuscaretta',
                 'Tuscaridium cygneum',
                 'Tuscaridium',
                 'Tuscarilla campanella',
                 'Tuscarilla nationalis',
                 'Tuscarilla similis',
                 'Tuscarilla',
                 'Tuscarora',
                 'Tuscaroridae'
                 )
            )
        ) AND a.observation_uuid = b.observation_uuid
    ) AND index_recorded_timestamp >= CAST('2001-01-01' AS datetime) 
      AND index_recorded_timestamp <= CAST('2001-12-31' AS datetime)
    """

In [5]:
## Query the database

# Get connection
conn = VARS.get_db_conn(dr, url, un, pw, name)

# Submit query
data = VARS.get_data(conn, sql)

# Close connection
conn.close()

In [10]:
## Show data

data[0].columns = data[1]
data = data[0]
df.head(2)

,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-04-11 04:20:30,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,373.500000,19.907844,-156.077373,3.26,181.800003,34.161999,8.972,urn:tid:mbari.org:T0288-02,Tiburon 0288,David Clague
1,2001-04-22 18:43:36,9CCB4C2F-B737-4E9F-AC0A-2CB3429551B9,rock,ROV,tiburon,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1567.400024,21.310648,-157.021308,1.77,322.700012,34.574001,2.680,urn:tid:mbari.org:T0300-01,Tiburon 0300,Gary Greene


## Pre-processing

In [12]:
## Drop duplicate rows that arise from associations, which we don't care about here

data = data.drop_duplicates()
print(data.shape)
data.head(2)

(172914, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-04-11 04:20:30,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,373.500000,19.907844,-156.077373,3.26,181.800003,34.161999,8.972,urn:tid:mbari.org:T0288-02,Tiburon 0288,David Clague
1,2001-04-22 18:43:36,9CCB4C2F-B737-4E9F-AC0A-2CB3429551B9,rock,ROV,tiburon,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1567.400024,21.310648,-157.021308,1.77,322.700012,34.574001,2.680,urn:tid:mbari.org:T0300-01,Tiburon 0300,Gary Greene


## Convert to DwC

In [13]:
## Start with basic event data and change col names

converted = data[['index_recorded_timestamp', 'video_sequence_name', 'observation_group', 'chief_scientist']]
converted = converted.rename(columns={
    'index_recorded_timestamp':'eventDate',
    'video_sequence_name':'eventID',
    'observation_group':'samplingProtocol',
    'chief_scientist':'recordedBy'
})
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-04-11 04:20:30,Tiburon 0288,ROV,David Clague
1,2001-04-22 18:43:36,Tiburon 0300,ROV,Gary Greene


A small number of records have samplingProtocol = NaN. However, they have one of the ROVs indicated in the eventID, so I feel comfortable forcing all records to ROV.

In [15]:
## Ensure samplingProtocol is always ROV

converted['samplingProtocol'] = 'ROV'
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-04-11 04:20:30,Tiburon 0288,ROV,David Clague
1,2001-04-22 18:43:36,Tiburon 0300,ROV,Gary Greene


In [16]:
## Remove whitespace from eventID

converted['eventID'] = [event.replace(' ', '_') for event in converted['eventID']]
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-04-11 04:20:30,Tiburon_0288,ROV,David Clague
1,2001-04-22 18:43:36,Tiburon_0300,ROV,Gary Greene


**Note** that this code also places an underscore between 'Doc' and 'Ricketts'. It's possible that using 'DocRicketts' could be preferable.

In [17]:
## Add datasetID, institutionCode

converted['datasetID'] = 'VARS-2001'
converted['institutionCode'] = 'MBARI'
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode
0,2001-04-11 04:20:30,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI
1,2001-04-22 18:43:36,Tiburon_0300,ROV,Gary Greene,VARS-2001,MBARI


In [18]:
## Format eventDate

formatted = []

for dt in converted['eventDate']:
    
    # Convert string to datetime
    try:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f') # some datetimes have milliseconds
    except ValueError:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        
    # Assign UTC timezone
#     utc = pytz.UTC
#     dt = dt.astimezone(utc)
    
    # Put in ISO format string
    dt = dt.isoformat()
    
    # Save in list
    formatted.append(dt + 'Z')

converted['eventDate'] = formatted
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI
1,2001-04-22T18:43:36Z,Tiburon_0300,ROV,Gary Greene,VARS-2001,MBARI


In [20]:
## Add in occurrence-related columns from data, renaming as needed

converted['occurrenceID'] = data['observation_uuid']
converted['scientificName'] = data['concept']
converted['occurrenceRemarks'] = data['concept']
converted['identifiedBy'] = data['observer']
converted['minimumDepthInMeters'] = round(data['depth_meters'], 1) - 2.2 # Sensor is at most 2 m shallower than camera/observed organism (?), and sensor is accurate within 20 cm
converted['maximumDepthInMeters'] = round(data['depth_meters'], 1) + 2.2
converted['verbatimDepth'] = round(data['depth_meters'], 1)
converted['decimalLatitude'] = data['latitude']
converted['decimalLongitude'] = data['longitude']
converted['dissolvedOxygenInMLPerL'] = data['oxygen_ml_per_l']
converted['pressureInPsi'] = data['psi']
converted['salinity'] = data['salinity']
converted['temperatureInCelsius'] = data['temperature_celsius']
converted['image_url'] = data['image_url']
converted['video_uri'] = data['video_uri']
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,maximumDepthInMeters,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,Actinopterygii,vars,...,375.7,373.5,19.907844,-156.077373,3.26,181.800003,34.161999,8.972,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0288-02
1,2001-04-22T18:43:36Z,Tiburon_0300,ROV,Gary Greene,VARS-2001,MBARI,9CCB4C2F-B737-4E9F-AC0A-2CB3429551B9,rock,rock,tiburon,...,1569.6,1567.4,21.310648,-157.021308,1.77,322.700012,34.574001,2.680,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0300-01


In [21]:
## Add coordinateUncertaintyInMeters 

converted['coordinateUncertaintyInMeters'] = 300

In [37]:
## Get a table containing all living concepts in VARS (deep sea guide)

living_concepts = pd.read_json('http://dsg.mbari.org/kb/v1/phylogeny/taxa/Eukaryota')
living_concepts = living_concepts[living_concepts['name'] != '']
living_concepts.head(2)

,name,rank
1,Abraliopsis,genus
2,Abraliopsis (Boreabraliopsis) felis,species


In [50]:
## Use living concepts to filter nonliving concepts out of data

converted = converted[converted['scientificName'].isin(living_concepts['name'].str.lower().tolist()) == True].copy()

In [60]:
## Make manual changes to scientific names based on notes below and create a list of unique names

converted['scientificName'] = converted['scientificName'].str.lower().str.strip()

manual_change_dic = {
    'teuthoidea': 'teuthida',
    'medusae': 'cnidaria',
    'lrj complex': 'rhopalonematidae',
    'neptunea-buccinum complex': 'buccinidae',
    'vitreosalpa gemini': 'salpidae',
    'sebastomus complex': 'sebastes(sebastomus)',
    'graneledoninae': 'octopodidae',
    'funiculina-balticina complex': 'funiculinidae'
}
converted['scientificName'].replace(manual_change_dic, inplace=True)

names = converted['scientificName'].unique()

There may be a more elegant way to filter out nonliving concepts...

In [61]:
## Look up names in WoRMS and save matched name, name ID and taxon ID to dicts ----- TAKES ~ 15 MINUTES TO DO ~ 700 NAMES

name_id_dic, name_dic, id_dic, class_dic = WoRMS.run_get_worms_from_scientific_name(names, verbose_flag=True)

Url didn't work for peniagone sp. a checking:  peniagone
Url didn't work for coryphaenoides armatus-yaquinae complex checking:  coryphaenoides
Url didn't work, check name:  hydromedusae
Url didn't work for bathyraja cf. microtrachys checking:  bathyraja
Url didn't work for grimpoteuthis sp. 5 checking:  grimpoteuthis
Url didn't work for umbellula sp. 2 checking:  umbellula
Url didn't work for coryphaenoides acrolepis-filifer complex checking:  coryphaenoides
Url didn't work for synallactidae gen. et sp. indet. checking:  synallactidae
Url didn't work for tomopterid eggcase checking:  tomopterid
Url didn't work, check name:  tomopterid
Url didn't work for opisthoteuthis sp. a checking:  opisthoteuthis
Url didn't work for cirrothauma sp. 1 checking:  cirrothauma
Url didn't work, check name:  sebastes(sebastomus)
Url didn't work for nuculana sp. a checking:  nuculana
Url didn't work for liparidae sp. 1 checking:  liparidae
Url didn't work for levinsenia sp. a checking:  levinsenia
Url did

A handful of names didn't match on WoRMS. Some of them I discussed previously with Nancy Jacobsen-Stout, so I know how to handle the problems. Others still need to be handled.
- teuthoidea (**This is an old name. Teuthida**)
- medusae (**An MBARI term for a jelly that cannot be identified, even to class level. Phylum Cnidaria.**)
- hydromedusae (**Nancy identified the lowest taxononimc level as Leptolida. WoRMS does not accept this; the accepted name is Hydroidolina. Not sure what to do here.**)
- lrj complex (**Little Red Jellies complex. Family Rhopalonematidae.**)
- neptunea-buccinum complex (**family Buccinidae**)
- tomopterid eggcase (**WoRMS will match Tomopteridae, is that correct?**)
- vitreosalpa gemini (**This is a special MBARI name for a species that has not been formally described yet. Family salpidae.**)
- sebastomus complex (**Sebastes (also sebastomus) is a subgenus of Sebastes. Sebastes(Sebastomus) should match on WoRMS; aphia id = 991848**)
- graneledoninae (**This is an instance where MBARI may be more up to date than WoRMS. Use family Octopodidae.**)
- funiculina-balticina complex (**Family Funiculinidae**)

In [62]:
## Create columns from WoRMS data

# Create scientificNameID column with the same content as scientificName
converted['scientificNameID'] = converted['scientificName'].str.strip().str.lower()

# Use dictionary to replace scientific names with name IDs
converted.replace({'scientificNameID':name_id_dic}, inplace=True)

# Repeat to create taxonID
converted['taxonID'] = converted['scientificName'].str.strip().str.lower()
converted.replace({'taxonID':id_dic}, inplace=True)

converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,actinopterygii,Actinopterygii,vars,...,-156.077373,3.26,181.800003,34.161999,8.972,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0288-02,300,urn:lsid:marinespecies.org:taxname:10194,10194
13,2001-03-28T04:56:42Z,Tiburon_0271,ROV,Bruce Robison,VARS-2001,MBARI,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,radiozoa,Radiozoa,schlin,...,-158.396073,1.49,64.000000,34.567001,3.045,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0271-09,300,urn:lsid:marinespecies.org:taxname:582421,582421


In [63]:
## Replace scientificName with matched scientific names from WoRMS

converted['scientificName'].replace(name_dic, inplace=True)
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,Actinopterygii,vars,...,-156.077373,3.26,181.800003,34.161999,8.972,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0288-02,300,urn:lsid:marinespecies.org:taxname:10194,10194
13,2001-03-28T04:56:42Z,Tiburon_0271,ROV,Bruce Robison,VARS-2001,MBARI,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,Radiozoa,Radiozoa,schlin,...,-158.396073,1.49,64.000000,34.567001,3.045,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0271-09,300,urn:lsid:marinespecies.org:taxname:582421,582421


In [64]:
## Create additional needed columns

converted['nameAccordingTo'] = 'WoRMS'
converted['occurrenceStatus'] = 'present'
converted['basisOfRecord'] = 'MachineObservation'

converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,Actinopterygii,vars,...,34.161999,8.972,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0288-02,300,urn:lsid:marinespecies.org:taxname:10194,10194,WoRMS,present,MachineObservation
13,2001-03-28T04:56:42Z,Tiburon_0271,ROV,Bruce Robison,VARS-2001,MBARI,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,Radiozoa,Radiozoa,schlin,...,34.567001,3.045,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0271-09,300,urn:lsid:marinespecies.org:taxname:582421,582421,WoRMS,present,MachineObservation


It is possible to add an associatedMedia column with the links to the image and video files as a pipe-delimited string. But I'm not going to do this for now. Instead, I'll drop extra rows that exist when images of the same occurence exist in multiple formats.

In [74]:
## Remove rows with duplicate occurrenceIDs

converted = converted.drop_duplicates(subset='occurrenceID', keep="first")
converted.shape

(117536, 27)

In [75]:
## Save columns for MeasurementOrFact file

mof = converted[['occurrenceID', 'dissolvedOxygenInMLPerL', 'pressureInPsi', 'salinity', 'temperatureInCelsius']].copy()

In [77]:
## Drop extra columns

converted = converted.drop(['image_url', 
                            'video_uri', 
                            'dissolvedOxygenInMLPerL', 
                            'pressureInPsi', 
                            'salinity', 
                            'temperatureInCelsius'], axis=1)
converted.head(2)

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,maximumDepthInMeters,verbatimDepth,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
0,2001-04-11T04:20:30Z,Tiburon_0288,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,Actinopterygii,vars,...,375.7,373.5,19.907844,-156.077373,300,urn:lsid:marinespecies.org:taxname:10194,10194,WoRMS,present,MachineObservation
13,2001-03-28T04:56:42Z,Tiburon_0271,ROV,Bruce Robison,VARS-2001,MBARI,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,Radiozoa,Radiozoa,schlin,...,1395.4,1393.2,21.474050,-158.396073,300,urn:lsid:marinespecies.org:taxname:582421,582421,WoRMS,present,MachineObservation


In [78]:
## Reorder columns - this just makes more sense to me and helps me keep track of things

converted = converted[['eventID', 
                       'eventDate', 
                       'samplingProtocol', 
                       'recordedBy', 
                       'datasetID', 
                       'institutionCode', 
                       'occurrenceID', 
                       'scientificName', 
                       'scientificNameID', 
                       'taxonID', 
                       'nameAccordingTo', 
                       'occurrenceStatus', 
                       'basisOfRecord', 
                       'identifiedBy', 
                       'occurrenceRemarks', 
                       'decimalLatitude', 
                       'decimalLongitude', 
                       'coordinateUncertaintyInMeters',
                       'minimumDepthInMeters', 
                       'maximumDepthInMeters', 
                       'verbatimDepth']]
converted.head(2)

,eventID,eventDate,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,scientificNameID,taxonID,...,occurrenceStatus,basisOfRecord,identifiedBy,occurrenceRemarks,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,minimumDepthInMeters,maximumDepthInMeters,verbatimDepth
0,Tiburon_0288,2001-04-11T04:20:30Z,ROV,David Clague,VARS-2001,MBARI,4A58D038-49AD-40C6-9C68-7366B55DB4D1,Actinopterygii,urn:lsid:marinespecies.org:taxname:10194,10194,...,present,MachineObservation,vars,Actinopterygii,19.907844,-156.077373,300,371.3,375.7,373.5
13,Tiburon_0271,2001-03-28T04:56:42Z,ROV,Bruce Robison,VARS-2001,MBARI,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,Radiozoa,urn:lsid:marinespecies.org:taxname:582421,582421,...,present,MachineObservation,schlin,Radiozoa,21.474050,-158.396073,300,1391.0,1395.4,1393.2


## Build MoF file

In [79]:
## Look at current structure

mof.head(2)

,occurrenceID,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius
0,4A58D038-49AD-40C6-9C68-7366B55DB4D1,3.26,181.800003,34.161999,8.972
13,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,1.49,64.000000,34.567001,3.045


In [80]:
## Convert to long format

mof_long = pd.melt(mof, id_vars='occurrenceID', var_name='measurementType', value_name='measurementValue')
mof_long.head(2)

,occurrenceID,measurementType,measurementValue
0,4A58D038-49AD-40C6-9C68-7366B55DB4D1,dissolvedOxygenInMLPerL,3.26
1,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,dissolvedOxygenInMLPerL,1.49


**Note** that it would be good to do more QC on these data. For example, there are several rows where salinity=0. Should these be NaN? Also, what is the appropriate rounding to use? I assume pressure isn't actually accurate to the 6th decimal place?

In [85]:
## Change measurementType names

mof_long['measurementType'].replace({
    'dissolvedOxygenInMLPerL':'dissolvedOxygen',
    'pressureInPsi':'pressure',
    'temperatureInCelsius':'temperature'
}, inplace = True)

**Note** that ideally standard vocabulary terms should be used here. You can use the NERC vocabulary server to [search](https://vocab.nerc.ac.uk/search_nvs/) for terms, but it can be a bit overwhelming. I just searched for "salinity" and realized that I don't know enough about how these data were collected to make an intelligent choice. 

In [86]:
## Add measurementUnit

mof_long['measurementUnit'] = 'mL per L seawater'
mof_long.loc[mof_long['measurementType'] == 'pressure', 'measurementUnit'] = 'psi'
mof_long.loc[mof_long['measurementType'] == 'salinity', 'measurementUnit'] = 'psu'
mof_long.loc[mof_long['measurementType'] == 'temperature', 'measurementUnit'] = 'celsius'

mof_long.head(2)

,occurrenceID,measurementType,measurementValue,measurementUnit
0,4A58D038-49AD-40C6-9C68-7366B55DB4D1,dissolvedOxygen,3.26,mL per L seawater
1,3F16C4C5-60CF-4A7C-A8A0-8E0FF59E488E,dissolvedOxygen,1.49,mL per L seawater


These two files - the occurrence file (converted) and the measurement or fact extension file (mof) - in addition to EML metadata would comprise a complete submission to OBIS.

I'd imagined running something like this for each year of data in VARS, and then concatenating the outcome for submission.